In [39]:
from __future__ import print_function
from pathlib import Path

In [40]:
# We'll need numpy for some mathematical operations
import numpy as np


# matplotlib for displaying the output
import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
%matplotlib inline


# and IPython.display for audio output
import IPython.display


# Librosa for audio
import librosa
# And the display module for visualization
import librosa.display

In [41]:
# Imports
# pandas
import pandas as pd
from pandas import Series,DataFrame
# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline

In [42]:
mp3dir = 'piano_first_thirty_sec'

In [43]:
#colnames = ['composer','description', 'symphony', 'file', 'part', 'tempo', 'bpm']
musicdf = pd.read_csv('./musicdf.csv', index_col=0, encoding='latin-1')

In [44]:
def load(audio_path):
    audio_path = '.\\'+ mp3dir + '\\' + audio_path
    my_file = Path(audio_path)
    if my_file.is_file():
        return librosa.load(audio_path, sr=None)
    else:
        return None, None

In [46]:
def getbpm(audio_path):
    y, sr = load(audio_path)
    onset_env = librosa.onset.onset_strength(y, sr=sr)
    return librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0]

In [47]:
def getDuration(audio_path):
    y, sr = load(audio_path)
    return librosa.core.get_duration(y,sr)

In [48]:
def getZeroCrossings(audio_path):
    y, sr = load(audio_path)
    return sum(librosa.zero_crossings(y, pad=False, threshold=0.001))

In [49]:
## http://musicinformationretrieval.com/spectral_features.html
## http://cs229.stanford.edu/proj2016/report/NiShiWugofski_FinalReport.pdf
def getAvgSpectralCentroid(audio_path):
    y, sr = load(audio_path)
    spectral_centroids = librosa.feature.spectral_centroid(y+0.01, sr=sr)[0]
    return spectral_centroids.mean()

def getAvgSpectralBW(audio_path):
    y, sr = load(audio_path)
    spectral_bandwidth_2 = librosa.feature.spectral_bandwidth(y+0.01, sr=sr)[0]
    return spectral_bandwidth_2.mean()

In [50]:
## http://musicinformationretrieval.com/chroma.html
hop_length = 512
def chroma(audio_path):
    y, sr = load(audio_path)
    chromagram = librosa.feature.chroma_cens(y, sr=sr, hop_length=hop_length)
    return pd.Series({
              "Chroma_C":chromagram[0].mean(), "Chroma_C#":chromagram[1].mean(),
              "Chroma_D":chromagram[2].mean(), "Chroma_D#":chromagram[3].mean(),
              "Chroma_E":chromagram[4].mean(), "Chroma_F":chromagram[5].mean(),
              "Chroma_F#":chromagram[6].mean(), "Chroma_G":chromagram[7].mean(),
              "Chroma_G#":chromagram[8].mean(), "Chroma_A":chromagram[9].mean(),
              "Chroma_A#":chromagram[10].mean(), "Chroma_B":chromagram[11].mean()})

In [51]:
musicdf['bpm'] = musicdf['file'].apply(getbpm)

In [52]:
musicdf['duration'] = musicdf['file'].apply(getDuration)

In [53]:
musicdf['zero_crossings'] = musicdf['file'].apply(getZeroCrossings)

In [54]:
musicdf['spectral_centroid_avg'] =  musicdf['file'].apply(getAvgSpectralCentroid)

In [55]:
musicdf['spectral_bw_avg'] =  musicdf['file'].apply(getAvgSpectralBW)

In [56]:
#musicdf = pd.concat([musicdf, musicdf['file'].apply(chroma)], axis=1)
musicdf.update(musicdf['file'].apply(chroma))

In [58]:
musicdf.to_csv('musicdf30.csv', encoding='latin-1')

In [57]:
musicdf.head()

,composer,description,symphony,file,part,tempo,bpm,duration,zero_crossings,spectral_centroid_avg,...,Chroma_B,Chroma_C,Chroma_C#,Chroma_D,Chroma_D#,Chroma_E,Chroma_F,Chroma_F#,Chroma_G,Chroma_G#
0,albeniz,Albeniz Isaac(1860-1909),Espana Opus 165,alb_esp1.mp3,Prelude,Andantino,120.185320,30.002698,33839,640.679613,...,0.141876,0.174059,0.235315,0.191668,0.270427,0.328634,0.232527,0.055501,0.050027,0.205815
1,albeniz,Albeniz Isaac(1860-1909),Espana Opus 165,alb_esp2.mp3,Tango,Andantino,120.185320,30.002698,26453,560.000413,...,0.161060,0.160480,0.317350,0.362431,0.250595,0.174810,0.172583,0.186237,0.170238,0.234457
2,albeniz,Albeniz Isaac(1860-1909),Espana Opus 165,alb_esp3.mp3,Malaguena,Allegretto,117.453835,30.002698,40915,761.211409,...,0.334959,0.281933,0.138918,0.170627,0.295441,0.301934,0.263001,0.335738,0.248279,0.130384
3,albeniz,Albeniz Isaac(1860-1909),Espana Opus 165,alb_esp4.mp3,Serenata,Allegretto,112.347147,30.002698,34665,678.501668,...,0.225666,0.225116,0.312408,0.398074,0.317425,0.131499,0.119087,0.200023,0.217149,0.185478
4,albeniz,Albeniz Isaac(1860-1909),Espana Opus 165,alb_esp5.mp3,Capricho Catalan,Allegretto,120.185320,30.002698,30595,583.487585,...,0.272851,0.067138,0.121657,0.338595,0.406299,0.223670,0.139772,0.165996,0.230227,0.194519
